In [11]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# GRADED FUNCTION: zero_pad

def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    ### START CODE HERE ### (≈ 1 line)
    X_pad = np.pad(X, ((0,0) ,(pad,pad) ,(pad,pad) ,(0,0)) ,'constant' , constant_values=(0,0))
    ### END CODE HERE ###
    
    return X_pad

In [13]:
# GRADED FUNCTION: conv_single_step

def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, result of convolving the sliding window (W, b) on a slice x of the input data
    """

    ### START CODE HERE ### (≈ 2 lines of code)
    # Element-wise product between a_slice and W. Do not add the bias yet.
    
    # A = (2,4,4,3) or any w = (2,2,3,4) where filter size of (2*2)*3channels of 4 filters 
    #matrix is taken in column matrix way in which each row is of 1st value of 3 channels ,followed by no of rows(height) ,
    # like this for next column matrix in same above size(width) this total is for no of examples(m=2)  
    #print('W.shape=',W.shape , 'W=',W)
    s = np.multiply(a_slice_prev,W) 
    #print('s.shape=',s.shape, 's=',s)  
    # Sum over all entries of the volume s.
    Z = np.sum(s)
    # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.
    Z = Z+b   # matrix multiply is elementwise of above form where at a time channels are also included 
    ### END CODE HERE ###

    return Z

In [9]:
# GRADED FUNCTION: conv_forward

def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    #print('A_prev.shape=' ,A_prev.shape ,'A_prev=',  A_prev)
    
    ### START CODE HERE ###
    # Retrieve dimensions from A_prev's shape (≈1 line)  
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape (≈1 line)
    (f, f, n_C_prev, n_C) = W.shape
    #print('W.shape=',W.shape , 'W=',W)
    
    # Retrieve information from "hparameters" (≈2 lines)
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    # Compute the dimensions of the CONV output volume using the formula given above. Hint: use int() to floor. (≈2 lines)
    n_H = int((n_H_prev - f + 2 * pad) / stride + 1)     
    
    n_W = int((n_W_prev - f + 2 * pad) / stride + 1)
    #print ('n_H=,n_W= ',n_H , n_W)
    
    # Initialize the output volume Z with zeros. (≈1 line)
    Z = np.zeros((m, n_H, n_W, n_C))
    #print('Z.shape=',Z.shape,'Z=' ,Z)
    
    # Create A_prev_pad by padding A_prev
    A_prev_pad = zero_pad(A_prev, pad)
    #print('A_prev_pad.shape=',A_prev_pad.shape , 'A_prev_pad=' , A_prev_pad)
    
    for i in range(m):                               # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i, :, :, :]            # Select ith training example's padded activation
        #print(a_prev_pad.shape)
        for h in range(n_H):                           # loop over vertical axis of the output volume
            for w in range(n_W):                        # loop over horizontal axis of the output volume
                for c in range(n_C):                        # loop over channels (= #filters) of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    a_slice_prev = a_prev_pad[vert_start: vert_end, horiz_start: horiz_end, :]
                    #print('a_slice_prev=',a_slice_prev)
                    #print(a_slice_prev.shape)
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈1 line)
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:,:,:,c], b[:,:,:,c])
                    print('Z.shape=',Z.shape , 'Z=', Z)
    
    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z,cache

In [18]:
np.random.seed(1)
A_prev = np.random.randn(2,4,4,3)
W = np.random.randn(2,2,3,3)
b = np.random.randn(1,1,1,3)
hparameters = {'pad':2 , 'stride':1}

z , cache_conv= conv_forward(A_prev, W, b, hparameters)
#print(z[3,2,1] , z)
print('final=',z.mean())
print(z)

Z.shape= (2, 7, 7, 3) Z= [[[[-1.23005814  0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.       

Z.shape= (2, 7, 7, 3) Z= [[[[-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.       

Z.shape= (2, 7, 7, 3) Z= [[[[-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]
   [-1.23005814  0.5505375   0.79280687]]

  [[-1.23005814  0.5505375   0.79280687]
   [ 0.51347407 -2.57869056  3.8533979 ]
   [-2.34826905  0.84478923 -1.16723166]
   [-0.47914362  1.15627822  1.13368337]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.       

Z.shape= (2, 7, 7, 3) Z= [[[[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [  5.13474074e-01  -2.57869056e+00   3.85339790e+00]
   [ -2.34826905e+00   8.44789235e-01  -1.16723166e+00]
   [ -4.79143620e-01   1.15627822e+00   1.13368337e+00]
   [  7.14313255e-01  -1.50983130e+00   2.84998890e-03]
   [ -3.62919311e+00   4.11435784e+00  -1.22380334e+00]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23901924e+00   1.55069517e+00  -8.50301106e-02]
   [ -6.28874831e+00  -8.74783701e-01   2.10498629e+00]
   [  0.00000000e+0

Z.shape= (2, 7, 7, 3) Z= [[[[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [  5.13474074e-01  -2.57869056e+00   3.85339790e+00]
   [ -2.34826905e+00   8.44789235e-01  -1.16723166e+00]
   [ -4.79143620e-01   1.15627822e+00   1.13368337e+00]
   [  7.14313255e-01  -1.50983130e+00   2.84998890e-03]
   [ -3.62919311e+00   4.11435784e+00  -1.22380334e+00]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23901924e+00   1.55069517e+00  -8.50301106e-02]
   [ -6.28874831e+00  -8.74783701e-01   2.10498629e+00]
   [ -5.90850902e-0

...

Z.shape= (2, 7, 7, 3) Z= [[[[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [  5.13474074e-01  -2.57869056e+00   3.85339790e+00]
   [ -2.34826905e+00   8.44789235e-01  -1.16723166e+00]
   [ -4.79143620e-01   1.15627822e+00   1.13368337e+00]
   [  7.14313255e-01  -1.50983130e+00   2.84998890e-03]
   [ -3.62919311e+00   4.11435784e+00  -1.22380334e+00]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23901924e+00   1.55069517e+00  -8.50301106e-02]
   [ -6.28874831e+00  -8.74783701e-01   2.10498629e+00]
   [ -5.90850902e-0

Z.shape= (2, 7, 7, 3) Z= [[[[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [  5.13474074e-01  -2.57869056e+00   3.85339790e+00]
   [ -2.34826905e+00   8.44789235e-01  -1.16723166e+00]
   [ -4.79143620e-01   1.15627822e+00   1.13368337e+00]
   [  7.14313255e-01  -1.50983130e+00   2.84998890e-03]
   [ -3.62919311e+00   4.11435784e+00  -1.22380334e+00]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23901924e+00   1.55069517e+00  -8.50301106e-02]
   [ -6.28874831e+00  -8.74783701e-01   2.10498629e+00]
   [ -5.90850902e-0

Z.shape= (2, 7, 7, 3) Z= [[[[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [  5.13474074e-01  -2.57869056e+00   3.85339790e+00]
   [ -2.34826905e+00   8.44789235e-01  -1.16723166e+00]
   [ -4.79143620e-01   1.15627822e+00   1.13368337e+00]
   [  7.14313255e-01  -1.50983130e+00   2.84998890e-03]
   [ -3.62919311e+00   4.11435784e+00  -1.22380334e+00]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23901924e+00   1.55069517e+00  -8.50301106e-02]
   [ -6.28874831e+00  -8.74783701e-01   2.10498629e+00]
   [ -5.90850902e-0

Z.shape= (2, 7, 7, 3) Z= [[[[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [  5.13474074e-01  -2.57869056e+00   3.85339790e+00]
   [ -2.34826905e+00   8.44789235e-01  -1.16723166e+00]
   [ -4.79143620e-01   1.15627822e+00   1.13368337e+00]
   [  7.14313255e-01  -1.50983130e+00   2.84998890e-03]
   [ -3.62919311e+00   4.11435784e+00  -1.22380334e+00]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23901924e+00   1.55069517e+00  -8.50301106e-02]
   [ -6.28874831e+00  -8.74783701e-01   2.10498629e+00]
   [ -5.90850902e-0

Z.shape= (2, 7, 7, 3) Z= [[[[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [  5.13474074e-01  -2.57869056e+00   3.85339790e+00]
   [ -2.34826905e+00   8.44789235e-01  -1.16723166e+00]
   [ -4.79143620e-01   1.15627822e+00   1.13368337e+00]
   [  7.14313255e-01  -1.50983130e+00   2.84998890e-03]
   [ -3.62919311e+00   4.11435784e+00  -1.22380334e+00]
   [ -1.23005814e+00   5.50537496e-01   7.92806866e-01]]

  [[ -1.23005814e+00   5.50537496e-01   7.92806866e-01]
   [ -1.23901924e+00   1.55069517e+00  -8.50301106e-02]
   [ -6.28874831e+00  -8.74783701e-01   2.10498629e+00]
   [ -5.90850902e-0